# 公司标签分布分析

分析 `output_production/company_tagging/company_tags.json` 中各维度标签的占比分布。


In [15]:
import json
from pathlib import Path
from collections import Counter

# 加载数据
TAGS_PATH = Path("output_production/company_tagging/company_tags.json")
with open(TAGS_PATH, "r", encoding="utf-8") as f:
    company_tags = json.load(f)

total_companies = len(company_tags)
print(f"总公司数: {total_companies}\n")

# 维度定义
TAG_DIMENSIONS = {
    "industry": "行业领域",
    "business_model": "商业模式",
    "target_market": "目标市场",
    "company_stage": "发展阶段",
    "tech_focus": "技术方向",
    "team_background": "团队背景",
}

def count_tags(companies: list, dimension: str) -> Counter:
    """统计某个维度的标签频次"""
    counter = Counter()
    for company in companies:
        value = company.get(dimension, [])
        if isinstance(value, list):
            counter.update(value)
        else:
            counter[value] += 1
    return counter

# 统计并打印各维度占比
for dim, dim_name in TAG_DIMENSIONS.items():
    counts = count_tags(company_tags, dim)
    print("=" * 50)
    print(f"【{dim_name}】({len(counts)} 种标签)")
    print("=" * 50)
    for tag, count in counts.most_common():
        pct = count / total_companies * 100
        bar = "█" * int(pct / 5) + "░" * (20 - int(pct / 5))
        print(f"  {tag:25s} {count:3d} ({pct:5.1f}%) {bar}")
    print()


总公司数: 138

【行业领域】(17 种标签)
  ai_llm                     75 ( 54.3%) ██████████░░░░░░░░░░
  enterprise_saas            55 ( 39.9%) ███████░░░░░░░░░░░░░
  content_media              33 ( 23.9%) ████░░░░░░░░░░░░░░░░
  consumer_hw                21 ( 15.2%) ███░░░░░░░░░░░░░░░░░
  cloud_infra                17 ( 12.3%) ██░░░░░░░░░░░░░░░░░░
  other                      16 ( 11.6%) ██░░░░░░░░░░░░░░░░░░
  robotics                   16 ( 11.6%) ██░░░░░░░░░░░░░░░░░░
  healthtech                 13 (  9.4%) █░░░░░░░░░░░░░░░░░░░
  social                     12 (  8.7%) █░░░░░░░░░░░░░░░░░░░
  edtech                     12 (  8.7%) █░░░░░░░░░░░░░░░░░░░
  fintech                    11 (  8.0%) █░░░░░░░░░░░░░░░░░░░
  ecommerce                  10 (  7.2%) █░░░░░░░░░░░░░░░░░░░
  semiconductor               9 (  6.5%) █░░░░░░░░░░░░░░░░░░░
  automotive                  7 (  5.1%) █░░░░░░░░░░░░░░░░░░░
  investment                  6 (  4.3%) ░░░░░░░░░░░░░░░░░░░░
  gaming                      5 (  3.6%) ░░░

In [16]:
# 汇总统计
print("=" * 50)
print("📊 汇总统计")
print("=" * 50)

# 每家公司平均标签数
multi_dims = ["industry", "business_model", "target_market", "tech_focus", "team_background"]
total_tags = sum(
    len(c.get(dim, [])) if isinstance(c.get(dim, []), list) else 1
    for c in company_tags
    for dim in multi_dims
)
print(f"每家公司平均标签数: {total_tags / total_companies:.1f}")

# 置信度统计
scores = [c.get("confidence_score", 0) for c in company_tags]
print(f"平均置信度: {sum(scores) / len(scores):.2f}")
print(f"置信度范围: {min(scores):.2f} - {max(scores):.2f}")

# 低置信度公司
low_conf = [(c["company_name"], c.get("confidence_score", 0)) for c in company_tags if c.get("confidence_score", 0) < 0.5]
if low_conf:
    print(f"\n⚠️ 低置信度公司 (<0.5): {len(low_conf)} 家")
    for name, score in sorted(low_conf, key=lambda x: x[1])[:5]:
        print(f"  - {name}: {score:.2f}")


📊 汇总统计
每家公司平均标签数: 11.7
平均置信度: 0.71
置信度范围: 0.00 - 0.90

⚠️ 低置信度公司 (<0.5): 7 家
  - 蚂蚁集团: 0.00
  - 闻声知意: 0.30
  - 月之暗面(Kimi): 0.35
  - Leadde. ai: 0.40
  - TensorStudio: 0.40


# 公司标签分布分析

分析 `output_production/company_tagging/company_tags.json` 中各维度标签的占比分布。


# 公司标签分布分析

分析 `output_production/company_tagging/company_tags.json` 中各维度标签的占比分布。


In [17]:
import json
from pathlib import Path
from collections import Counter

# 加载数据
TAGS_PATH = Path("output_production/company_tagging/company_tags.json")
with open(TAGS_PATH, "r", encoding="utf-8") as f:
    company_tags = json.load(f)

TOTAL = len(company_tags)
print(f"✓ 加载 {TOTAL} 家公司数据")


✓ 加载 138 家公司数据


In [18]:
# 定义标签维度
TAG_DIMENSIONS = {
    "industry": "行业领域",
    "business_model": "商业模式",
    "target_market": "目标市场",
    "company_stage": "发展阶段",
    "tech_focus": "技术方向",
    "team_background": "团队背景",
}

def count_tags(companies: list, dimension: str) -> Counter:
    """统计某个维度的标签频次"""
    counter = Counter()
    for company in companies:
        value = company.get(dimension, [])
        if isinstance(value, list):
            counter.update(value)
        else:
            counter[value] += 1
    return counter

def print_dimension_summary(dim: str, dim_name: str, counter: Counter, total: int):
    """打印某个维度的占比汇总"""
    print(f"\n{'='*60}")
    print(f"【{dim_name}】({len(counter)} 种标签)")
    print(f"{'='*60}")
    
    for tag, count in counter.most_common():
        pct = count / total * 100
        bar = "█" * int(pct / 2)  # 简单的文字进度条
        print(f"  {tag:<25} {count:>3} 家 ({pct:>5.1f}%) {bar}")

# 统计并打印所有维度
for dim, dim_name in TAG_DIMENSIONS.items():
    counter = count_tags(company_tags, dim)
    print_dimension_summary(dim, dim_name, counter, TOTAL)



【行业领域】(17 种标签)
  ai_llm                     75 家 ( 54.3%) ███████████████████████████
  enterprise_saas            55 家 ( 39.9%) ███████████████████
  content_media              33 家 ( 23.9%) ███████████
  consumer_hw                21 家 ( 15.2%) ███████
  cloud_infra                17 家 ( 12.3%) ██████
  other                      16 家 ( 11.6%) █████
  robotics                   16 家 ( 11.6%) █████
  healthtech                 13 家 (  9.4%) ████
  social                     12 家 (  8.7%) ████
  edtech                     12 家 (  8.7%) ████
  fintech                    11 家 (  8.0%) ███
  ecommerce                  10 家 (  7.2%) ███
  semiconductor               9 家 (  6.5%) ███
  automotive                  7 家 (  5.1%) ██
  investment                  6 家 (  4.3%) ██
  gaming                      5 家 (  3.6%) █
  biotech                     2 家 (  1.4%) 

【商业模式】(8 种标签)
  b2b                        91 家 ( 65.9%) ████████████████████████████████
  platform                   82 家 ( 59.

In [19]:
# 综合统计
print(f"\n{'='*60}")
print("📊 综合统计")
print(f"{'='*60}")
print(f"\n总公司数: {TOTAL}")

# 统计每家公司的标签数
multi_dims = ["industry", "business_model", "target_market", "tech_focus", "team_background"]
tag_counts_per_company = []
for c in company_tags:
    total_tags = sum(len(c.get(d, [])) if isinstance(c.get(d, []), list) else 1 for d in multi_dims)
    tag_counts_per_company.append(total_tags)

avg_tags = sum(tag_counts_per_company) / len(tag_counts_per_company)
print(f"每家公司平均标签数: {avg_tags:.1f}")
print(f"标签数范围: {min(tag_counts_per_company)} - {max(tag_counts_per_company)}")

# 置信度统计
confidence_scores = [c.get("confidence_score", 0) for c in company_tags]
avg_conf = sum(confidence_scores) / len(confidence_scores)
print(f"\n平均置信度: {avg_conf:.2f}")
print(f"置信度范围: {min(confidence_scores):.2f} - {max(confidence_scores):.2f}")

# 低置信度公司
low_conf = [(c["company_name"], c.get("confidence_score", 0)) 
            for c in company_tags if c.get("confidence_score", 0) < 0.5]
if low_conf:
    print(f"\n⚠️ 低置信度公司 (<0.5): {len(low_conf)} 家")
    for name, score in sorted(low_conf, key=lambda x: x[1])[:5]:
        print(f"  - {name}: {score:.2f}")



📊 综合统计

总公司数: 138
每家公司平均标签数: 11.7
标签数范围: 0 - 22

平均置信度: 0.71
置信度范围: 0.00 - 0.90

⚠️ 低置信度公司 (<0.5): 7 家
  - 蚂蚁集团: 0.00
  - 闻声知意: 0.30
  - 月之暗面(Kimi): 0.35
  - Leadde. ai: 0.40
  - TensorStudio: 0.40
